# Importing necesaary libraries and packages

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm.notebook import tqdm
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

# Importing necessary datasets and arrange them into Train, test,and Validation usecases

In [2]:
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
# train2 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv")
# train2['toxic'] = train2.toxic.round().astype(int)

valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
# sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [3]:
def def2(texts, tokenizer,maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts.tolist(), 
        return_token_type_ids=False,
        pad_to_max_length=True,        
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

# Buiding a sequential model using Accuracy as metrics and Binary_Crossentropy as Loss function. 

In [4]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Start the TPU Accelerator!

In [5]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ",strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [6]:
AUTO = tf.data.experimental.AUTOTUNE

In [28]:
epochs = 5
batch_size = 16 * strategy.num_replicas_in_sync
max_len = 192
MODEL='../input/jplu-tf-xlm-roberta-large'
# MODEL1 = 'xlmr.base'

In [29]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [15]:
train = pd.concat([
    train1[['comment_text', 'toxic']],
#     train2[['comment_text', 'toxic']].query('toxic==1'),
#     train2[['comment_text', 'toxic']].query('toxic==0').sample(n=100000)
])

In [16]:
x_train = def2(train.comment_text.values, tokenizer,maxlen=max_len)
x_valid = def2(valid.comment_text.values, tokenizer,maxlen=max_len)
x_test = def2(test.content.values, tokenizer,maxlen=max_len)

y_train = train['toxic'].values
y_valid = valid['toxic'].values

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [30]:
xlmr_model = TFAutoModel.from_pretrained(MODEL)

Some layers from the model checkpoint at ../input/jplu-tf-xlm-roberta-large were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at ../input/jplu-tf-xlm-roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


In [31]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(AUTO)
)

validation_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(batch_size)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(batch_size)
)

In [32]:
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=192)

Some layers from the model checkpoint at ../input/jplu-tf-xlm-roberta-large were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaModel were initialized from the model checkpoint at ../input/jplu-tf-xlm-roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


In [33]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tfxlm_roberta_model_1 (TFXLM TFBaseModelOutputWithPool 559890432 
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________


# Training my XLM-Roberta_Model with english only data.

Note: It took me more than 15 minutes for single epoch training.

In [35]:
steps = x_train.shape[0] // batch_size
train_history = model.fit(
    train_dataset,
    steps_per_epoch=steps,
    validation_data=validation_dataset,
    epochs=1,
)

TypeError: fit() got an unexpected keyword argument 'test_data'

In [ ]:
output = model.predict(test_dataset, verbose=1)

# Training my model with multiple_language data!

In [ ]:
steps = x_valid.shape[0] // batch_size
train_history_2 = model.fit(
    validation_dataset.repeat(),
    steps_per_epoch=steps,
    epochs=10
)

In [ ]:
output = model.predict(test_dataset, verbose=1)
# sub.to_csv('submission.csv', index=False)

172/499 [=========>....................] - ETA: 47s